#Aulas 11 e 12

## GAN

In [ ]:
# gan_completo5.py
# import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# example of training a gan on mnist
from numpy import expand_dims, zeros, ones, vstack
from numpy.random import randn, randint
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, \
  LeakyReLU, Dropout, MaxPooling2D, UpSampling2D
from matplotlib import pyplot
import sys

def define_discriminator(in_shape=(28,28,1)):
  model = Sequential() # 28x28
  model.add(Conv2D(20, kernel_size=(5,5), activation='relu',
    input_shape=in_shape )) #20x24x24
  model.add(MaxPooling2D(pool_size=(2,2))) #20x12x12
  model.add(Conv2D(40, kernel_size=(5,5), activation='relu')) #40x8x8
  model.add(MaxPooling2D(pool_size=(2,2))) #40x4x4
  model.add(Flatten()) #640
  model.add(Dense(100, activation='relu')) #100
  model.add(Dense(1, activation='sigmoid')) #1
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

def define_generator(latent_dim):
  model = Sequential()
  model.add(Dense(64 * 7 * 7, input_dim=latent_dim,activation="relu"))
  model.add(Reshape((7, 7, 64)))
  model.add(UpSampling2D(size = (2,2))) #14x14
  model.add(Conv2D(64, 2, activation = 'relu', padding = 'same'))
  model.add(UpSampling2D(size = (2,2))) #28x28
  model.add(Conv2D( 8, 2, activation = 'relu', padding = 'same'))
  model.add(Conv2D( 1, 1, activation = 'sigmoid', padding = 'same'))
  return model

def define_gan(g_model, d_model):
  # make weights in the discriminator not trainable
  d_model.trainable = False
  model = Sequential()
  model.add(g_model)
  model.add(d_model)
  # Muda os nomes dos layers
  model.layers[0]._name="gerador"
  model.layers[1]._name="discriminador"
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

def load_real_samples():
  (trainX, trainy), (_, _) = load_data()
  trainX = trainX[(trainy==7)]
  del trainy
  X = expand_dims(trainX, axis=-1)
  X = X.astype('float32')
  X = X / 255.0
  return X

def generate_real_samples(dataset, n_samples):
  ix = randint(0, dataset.shape[0], n_samples)
  X = dataset[ix]
  y = ones((n_samples, 1))
  return X, y

def generate_latent_points(latent_dim, n_samples):
  x_input = randn(n_samples, latent_dim)
  return x_input

def generate_fake_samples(g_model, latent_dim, n_samples):
  x_input = generate_latent_points(latent_dim, n_samples)
  X = g_model.predict(x_input)
  y = zeros((n_samples, 1))
  return X, y

def save_plot(examples, epoch, n=5):
  for i in range(n * n):
    pyplot.subplot(n, n, 1 + i)
    pyplot.axis('off')
    pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
  filename = 'generated_plot_e%03d.png' % (epoch+1)
  pyplot.savefig(filename)
  pyplot.close()

def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
  X_real, y_real = generate_real_samples(dataset, n_samples)
  _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
  print('Acuracia do discriminador em amostras reais: %.0f%%, falsas: %.0f%%' % (acc_real*100, acc_fake*100))
  save_plot(x_fake, epoch)

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=100):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  # manually enumerate epochs
  for i in range(n_epochs): #i==epoch
    # enumerate batches over the training set
    for j in range(bat_per_epo):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
      d_loss, d_accuracy = d_model.train_on_batch(X, y)
      X_gan = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, y_gan)
      if (j+1==bat_per_epo):
        print('epoch=%d, batch=%d/%d, d_acc=%.3f, g_loss=%.3f (bin_crossentr)'
          % (i+1, j+1, bat_per_epo, d_accuracy, g_loss))
    if (i+1) % 10 == 0:
      summarize_performance(i, g_model, d_model, dataset, latent_dim)
    if i+1==n_epochs:
      filename = 'generator_model_%03d.h5' % (i + 1)
      g_model.save(filename)

# size of the latent space
latent_dim = 10
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
save_plot(dataset,-1)
train(g_model, d_model, gan_model, dataset, latent_dim)

A saída de streaming foi truncada nas últimas 5000 linhas.
2/2 [==============================] - 0s 7ms/step
epoch=21, batch=62/62, d_acc=0.630, g_loss=0.999 (bin_crossentr)
2/2 [==============================] - 0s 11ms/step
epoch=22, batch=62/62, d_acc=0.650, g_loss=0.711 (bin_crossentr)
2/2 [==============================] - 0s 10ms/step
epoch=23, batch=62/62, d_acc=0.730, g_loss=0.721 (bin_crossentr)
2/2 [==============================] - 0s 11ms/step
epoch=24, batch=62/62, d_acc=0.660, g_loss=0.723 (bin_crossentr)
2/2 [==============================] - 0s 12ms/step
epoch=25, batch=62/62, d_acc=0.580, g_loss=1.003 (bin_crossentr)
2/2 [==============================] - 0s 11ms/step
epoch=26, batch=62/62, d_acc=0.630, g_loss=0.731 (bin_crossentr)
2/2 [==============================] - 0s 4ms/step
epoch=27, batch=62/62, d_acc=0.550, g_loss=0.649 (bin_crossentr)
2/2 [==============================] - 0s 5ms/step
epoch=28, batch=62/62, d_acc=0.670, g_loss=0.758 (bin_crossentr)
2/2 [===

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Lição de casa

In [ ]:
# gan_completo5.py
# import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# example of training a gan on mnist
from numpy import expand_dims, zeros, ones, vstack
from numpy.random import randn, randint
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from keras.datasets import cifar10, mnist, fashion_mnist
from keras.preprocessing import image
from keras.models import Model, Sequential, load_model
from keras.utils import to_categorical
from keras import optimizers
import keras
import numpy
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, \
  LeakyReLU, Dropout, MaxPooling2D, UpSampling2D
from matplotlib import pyplot
import sys

def define_discriminator(in_shape=(32,32,3)):
  model = Sequential() # 28x28
  model.add(Conv2D(20, kernel_size=(5,5), activation='relu',
    input_shape=in_shape )) #20x24x24
  model.add(MaxPooling2D(pool_size=(2,2))) #20x12x12
  model.add(Conv2D(40, kernel_size=(5,5), activation='relu')) #40x8x8
  model.add(MaxPooling2D(pool_size=(2,2))) #40x4x4
  model.add(Flatten()) #640
  model.add(Dense(100, activation='relu')) #100
  model.add(Dense(1, activation='sigmoid')) #1
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

def define_generator(latent_dim):
  model = Sequential()
  model.add(Dense(64 * 8 * 8, input_dim=latent_dim,activation="relu"))
  model.add(Reshape((8, 8, 64)))
  model.add(UpSampling2D(size = (2,2))) #14x14
  model.add(Conv2D(64, 2, activation = 'relu', padding = 'same'))
  model.add(UpSampling2D(size = (2,2))) #28x28
  model.add(Conv2D( 8, 2, activation = 'relu', padding = 'same'))
  model.add(Conv2D( 3, 3, activation = 'sigmoid', padding = 'same'))
  return model

def define_gan(g_model, d_model):
  # make weights in the discriminator not trainable
  d_model.trainable = False
  model = Sequential()
  model.add(g_model)
  model.add(d_model)
  # Muda os nomes dos layers
  model.layers[0]._name="gerador"
  model.layers[1]._name="discriminador"
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

def load_real_samples():
  (trainX, trainy), (_, _) = cifar10.load_data()
  ind = numpy.where(numpy.isin(trainy, [1]))[0]
  trainX = trainX[ind]
  del trainy
  X = trainX
  X = X.astype('float32')
  X = X / 255.0
  return X

def generate_real_samples(dataset, n_samples):
  ix = randint(0, dataset.shape[0], n_samples)
  X = dataset[ix]
  y = ones((n_samples, 1))
  return X, y

def generate_latent_points(latent_dim, n_samples):
  x_input = randn(n_samples, latent_dim)
  return x_input

def generate_fake_samples(g_model, latent_dim, n_samples):
  x_input = generate_latent_points(latent_dim, n_samples)
  X = g_model.predict(x_input)
  y = zeros((n_samples, 1))
  return X, y

def save_plot(examples, epoch, n=5):
  for i in range(n * n):
    pyplot.subplot(n, n, 1 + i)
    pyplot.axis('off')
    pyplot.imshow(examples[i, :, :, :])
  filename = 'generated_plot_e%03d.png' % (epoch+1)
  pyplot.savefig(filename)
  pyplot.close()

def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
  X_real, y_real = generate_real_samples(dataset, n_samples)
  _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
  print('Acuracia do discriminador em amostras reais: %.0f%%, falsas: %.0f%%' % (acc_real*100, acc_fake*100))
  save_plot(x_fake, epoch)

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=100):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  # manually enumerate epochs
  for i in range(n_epochs): #i==epoch
    # enumerate batches over the training set
    for j in range(bat_per_epo):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      print(X_real.shape, X_fake.shape)
      X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
      d_loss, d_accuracy = d_model.train_on_batch(X, y)
      X_gan = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, y_gan)
      if (j+1==bat_per_epo):
        print('epoch=%d, batch=%d/%d, d_acc=%.3f, g_loss=%.3f (bin_crossentr)'
          % (i+1, j+1, bat_per_epo, d_accuracy, g_loss))
    if (i+1) % 10 == 0:
      summarize_performance(i, g_model, d_model, dataset, latent_dim)
    if i+1==n_epochs:
      filename = 'generator_model_%03d.h5' % (i + 1)
      g_model.save(filename)

# size of the latent space
latent_dim = 10
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
save_plot(dataset,-1)
train(g_model, d_model, gan_model, dataset, latent_dim)

A saída de streaming foi truncada nas últimas 5000 linhas.
2/2 [==============================] - 0s 8ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 4ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 4ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 4ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 6ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 4ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 5ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 5ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 17ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 3ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [==============================] - 0s 3ms/step
(50, 32, 32, 3) (50, 32, 32, 3)
2/2 [======================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# gan_completo5.py
# import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# example of training a gan on mnist
from numpy import expand_dims, zeros, ones, vstack
from numpy.random import randn, randint
import tensorflow
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from keras.datasets import cifar10, mnist, fashion_mnist
from keras.preprocessing import image
from keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import BatchNormalization
from keras.utils import to_categorical
from keras import optimizers
import keras
import numpy
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, \
  LeakyReLU, Dropout, MaxPooling2D, UpSampling2D
from matplotlib import pyplot
import sys

def define_discriminator(in_shape=(32,32,3)):
  model = Sequential() # 28x28
  model.add(Conv2D(20, kernel_size=(5,5), activation='relu',
    input_shape=in_shape )) #20x24x24
  model.add(MaxPooling2D(pool_size=(2,2))) #20x12x12
  model.add(Conv2D(40, kernel_size=(5,5), activation='relu')) #40x8x8
  model.add(MaxPooling2D(pool_size=(2,2))) #40x4x4
  model.add(Flatten()) #640
  model.add(Dense(100, activation='relu')) #100
  model.add(Dense(1, activation='sigmoid')) #1
  model.add(BatchNormalization())
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

def define_generator(latent_dim):
  model = Sequential()
  model.add(Dense(64 * 8 * 8, input_dim=latent_dim,activation="relu"))
  model.add(BatchNormalization())
  model.add(Reshape((8, 8, 64)))
  model.add(UpSampling2D(size = (2,2))) #14x14
  model.add(Conv2D(64, 2, activation = 'relu', padding = 'same'))
  model.add(UpSampling2D(size = (2,2))) #28x28
  model.add(Conv2D( 8, 2, activation = 'relu', padding = 'same'))
  model.add(Conv2D( 3, 3, activation = 'sigmoid', padding = 'same'))
  return model

def define_gan(g_model, d_model):
  # make weights in the discriminator not trainable
  d_model.trainable = False
  model = Sequential()

  model.add(g_model)
  model.add(d_model)
  # Muda os nomes dos layers
  model.layers[0]._name="gerador"
  model.layers[1]._name="discriminador"
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

def load_real_samples():
  (trainX, trainy), (_, _) = cifar10.load_data()
  ind = numpy.where(numpy.isin(trainy, [1]))[0]
  trainX = trainX[ind]
  del trainy
  X = trainX
  X = X.astype('float32')
  X = X / 255.0
  return X

def generate_real_samples(dataset, n_samples):
  ix = randint(0, dataset.shape[0], n_samples)
  X = dataset[ix]
  y = ones((n_samples, 1))
  return X, y

def generate_latent_points(latent_dim, n_samples):
  x_input = randn(n_samples, latent_dim)
  return x_input

def generate_fake_samples(g_model, latent_dim, n_samples):
  x_input = generate_latent_points(latent_dim, n_samples)
  X = g_model.predict(x_input)
  y = zeros((n_samples, 1))
  return X, y

def save_plot(examples, epoch, n=5):
  for i in range(n * n):
    pyplot.subplot(n, n, 1 + i)
    pyplot.axis('off')
    pyplot.imshow(examples[i, :, :, :])
  filename = 'generated_plot22_e%03d.png' % (epoch+1)
  pyplot.savefig(filename)
  pyplot.close()

def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
  X_real, y_real = generate_real_samples(dataset, n_samples)
  _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
  print('Acuracia do discriminador em amostras reais: %.0f%%, falsas: %.0f%%' % (acc_real*100, acc_fake*100))
  save_plot(x_fake, epoch)

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=100):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  # manually enumerate epochs
  for i in range(n_epochs): #i==epoch
    # enumerate batches over the training set
    for j in range(bat_per_epo):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      print(X_real.shape, X_fake.shape)
      X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
      d_loss, d_accuracy = d_model.train_on_batch(X, y)
      X_gan = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, y_gan)
      if (j+1==bat_per_epo):
        print('epoch=%d, batch=%d/%d, d_acc=%.3f, g_loss=%.3f (bin_crossentr)'
          % (i+1, j+1, bat_per_epo, d_accuracy, g_loss))
    if (i+1) % 10 == 0:
      summarize_performance(i, g_model, d_model, dataset, latent_dim)
    if i+1==n_epochs:
      filename = 'generator_model22_%03d.h5' % (i + 1)
      g_model.save(filename)

# size of the latent space
latent_dim = 10
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
save_plot(dataset,-1)
train(g_model, d_model, gan_model, dataset, latent_dim)

Exception ignored in: <function _get_module_lock.<locals>.cb at 0x7cc6011c2200>
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 207, in cb
KeyboardInterrupt: 


170498071/170498071 [==============================] - 2s 0us/step


KeyboardInterrupt: ignored